# Wrangling Data (cleansing)

## Statistics methods versus NaN

**NOTE**: numpy.nan, if present in array even once, makes all statistics to numpy.nan.   
> In case of MATLAB, mean/sum results in NaN, but max/min ignores NaN and return value from non-NaN elements

In [14]:
import numpy as np 
a = np.arange(10.)
print(a, a.mean(), a.sum(), a.min(), a.max())

a[9]=np.nan;
print(a, a.mean(), a.sum(), a.min(), a.max())

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.] 4.5 45.0 0.0 9.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8. nan] nan nan nan nan


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


**NOTE**: With pandas, statistics methods of Series/DataFrame ignores numpy.nan! 

In [24]:
import pandas as pd
s = pd.Series( np.arange(10.))
print(s.values, s.mean(), s.sum(), s.min(), s.max())
s[9] = np.nan;
print(s.values, s.mean(), s.sum(), s.min(), s.max())

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.] 4.5 45.0 0.0 9.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8. nan] 4.0 36.0 0.0 8.0


## Locating NA

In [106]:
df = pd.DataFrame({"A":np.array([1,2,3,np.nan,5])})
df.A.isnull()

0    False
1    False
2    False
3     True
4    False
Name: A, dtype: bool

## Filling NA

In [39]:
# Fill by median
df = pd.DataFrame({'my_feature': np.arange(10.)})
df.my_feature[5] = np.nan
df.my_feature.fillna( df.my_feature.mean(), inplace=True )  # inplace = ovwerrite original
df

,my_feature
0,0.000000
1,1.000000
2,2.000000
3,3.000000
4,4.000000
5,4.444444
6,6.000000
7,7.000000
8,8.000000
9,9.000000


In [41]:
# Fill by fixed value
df = pd.DataFrame({'my_feature': np.arange(10.)})
df.my_feature[5] = np.nan
df.my_feature.fillna( 0, inplace=True)  # inplace = ovwerrite original
df

,my_feature
0,0.0
1,1.0
2,2.0
3,3.0
4,4.0
5,0.0
6,6.0
7,7.0
8,8.0
9,9.0


In [43]:
# Fill by polynomial interpolation (1st order)
df = pd.DataFrame({'my_feature': np.arange(10.)})
df.my_feature[5] = np.nan
df.my_feature.interpolate( method='polynomial', order=1, inplace=True)  # inplace = ovwerrite original
df

,my_feature
0,0.0
1,1.0
2,2.0
3,3.0
4,4.0
5,5.0
6,6.0
7,7.0
8,8.0
9,9.0


## Dropping Data

In [64]:
# Drop Row(s) having NA in it 
df = pd.DataFrame({'my_feature': np.arange(5.)})
df.my_feature[3] = np.nan
df = df.dropna() # default is axis=0: any row
df.reset_index(drop=True,inplace=True)  # reassign index 
df

,my_feature
0,0.0
1,1.0
2,2.0
3,4.0


In [61]:
# Drop Column(s) having NA in it 
df = pd.DataFrame({'my_feature': np.arange(5.), 'my_feature2': np.arange(5.)})
df.my_feature[3] = np.nan
df = df.dropna(axis=1) # default is axis=0: any row
df

,my_feature2
0,0.0
1,1.0
2,2.0
3,3.0
4,4.0


In [65]:
# Drop rows having more han 1 NAN
df = pd.DataFrame()
df['A'] = np.arange(5.);
df['B'] = np.arange(5.);
df.loc[2,'A'] = df.loc[2,'B'] = np.nan;
df.loc[3,'A'] = np.nan;
df.dropna(axis=0, thresh=1, inplace=True)  # if count(NaN) > tresh then drop 
df.reset_index(drop=True,inplace=True)  # reassign index 
df

,A,B
0,0.0,0.0
1,1.0,1.0
2,NaN,3.0
3,4.0,4.0


In [66]:
# Drop columns by names 
df = pd.DataFrame()
df['A'] = np.arange(5.);
df['B'] = np.arange(5.);
df['C'] = np.arange(5.);
df.drop( labels=['A','C'], axis=1, inplace=True)
df

,B
0,0.0
1,1.0
2,2.0
3,3.0
4,4.0


In [79]:
# Drop duplicated rows in the specified features (columns) 
df = pd.DataFrame() 
df['A'] = np.array([1, 2, 1, 3, 1])
df['B'] = np.array([10,20,10,30,100])
df1 = df.drop_duplicates(subset=['A'])  # duplication in A column removed
df1.reset_index(drop=True, inplace=True) # re-create index
print(df1,'\n')
df2 = df.drop_duplicates(subset=['A','B']) # cuplication in A "AND" B columns removed
df2.reset_index(drop=True, inplace=True)   # re-create index 
print(df2)

   A   B
0  1  10
1  2  20
2  3  30 

   A    B
0  1   10
1  2   20
2  3   30
3  1  100


In [85]:
# Chain of drops 
df = pd.DataFrame() 
df['A'] = np.array([1, 2, 1, 3, 1])
df['B'] = np.array([10,20,10,30,100])
df['C'] = np.array([np.nan, 20, 30, 40, 50])

df = df.dropna(axis=0).drop(labels=['C'], axis=1).drop_duplicates(subset=['A']).reset_index(drop=True)
df

,A,B
0,2,20
1,1,10
2,3,30


# More Wrangling

## Change data type

In [96]:
df = pd.DataFrame()
df['A'] = ['1','2','3','4','5']
df['B'] = ['1.1','2.2','3.3','4.4','5.5']
print( df.dtypes, '\n' )

df.A = pd.to_numeric(df.A, errors='coerce')  # error = [ raise(default) | coerce | ignore ]
df.B = pd.to_numeric(df.B, errors='coerce')
print( df.dtypes )

A    object
B    object
dtype: object 

A      int64
B    float64
dtype: object


## String Manipulation and Lambda Function

In [123]:
df = pd.DataFrame()
df['FirstName'] = ['AAA', 'BBB', 'CCC', 'DDD ddd']
df['LastName'] = ['aaa', 'bbb', 'ccc', np.nan]

# locate FirstName contains space, and LastName is NaN 
selector = df.FirstName.str.contains(' ') & df.LastName.isnull()
df.loc[selector,'LastName'] = df.FirstName[selector].apply( lambda x: x.split(' ')[1])
df.loc[selector,'FirstName'] = df.FirstName[selector].apply( lambda x: x.split(' ')[0])
df

,FirstName,LastName
0,AAA,aaa
1,BBB,bbb
2,CCC,ccc
3,DDD,ddd


In [132]:
import re

# Remove unwanted character 
df = pd.DataFrame() 
df['Weight'] = ['10','20','30kg', '40']

df.Weight = df.Weight.apply( lambda x: re.sub('[^0-9]', '', str(x)))  #  in str(x), substitude (non-numerical) => ('') 
df.Weight = pd.to_numeric(df.Weight, errors='coerce') # to numeric 
df

,Weight
0,10
1,20
2,30
3,40


## OMAKE (examine feature statistics)

In [100]:
df = pd.DataFrame()
df['Age'] = np.array([7,22,33,22,40,22,40,33,27])
print('unique = ', df.Age.unique())
print('value_counts = \n', df.Age.value_counts())

unique =  [ 7 22 33 40 27]
value_counts = 
 22    3
40    2
33    2
27    1
7     1
Name: Age, dtype: int64
